In [1]:
import gpflow
import numpy as np
import math
import tensorflow as tf
import sys

import pickle

In [5]:
#{'processed_history':ad,'g0_index':0,'a_f0_index':7}
root =  '../../Downloads/distributions/'
with open('{}parsed_to_phi.pkl'.format(root),'rb') as f:
    ad = pickle.load(f)

In [ ]:
ad.keys()

In [ ]:

#ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
#sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)


In [ ]:
import get_history as gh

In [3]:
import feather
import pandas as pd

In [7]:
def gather_cols(params, indices, name=None):
    """Gather columns of a 2D tensor.

    Args:
        params: A 2D tensor.
        indices: A 1D tensor. Must be one of the following types: ``int32``, ``int64``.
        name: A name for the operation (optional).

    Returns:
        A 2D Tensor. Has the same type as ``params``.
    """
    with tf.op_scope([params, indices], name, "gather_cols") as scope:
        # Check input
        params = tf.convert_to_tensor(params, name="params")
        indices = tf.convert_to_tensor(indices, name="indices")
        try:
            params.get_shape().assert_has_rank(2)
        except ValueError:
            raise ValueError('\'params\' must be 2D.')
        try:
            indices.get_shape().assert_has_rank(1)
        except ValueError:
            raise ValueError('\'params\' must be 1D.')

        # Define op
        p_shape = tf.shape(params)
        p_flat = tf.reshape(params, [-1])
        i_flat = tf.reshape(tf.reshape(tf.range(0, p_shape[0]) * p_shape[1],
                                       [-1, 1]) + indices, [-1])
        return tf.reshape(tf.gather(p_flat, i_flat),
                          [p_shape[0], -1])
    

In [8]:
def get_theta(dim_baseline):
    m = np.eye(dim_baseline)
    m = np.add(m,.1)
    return m
    

In [6]:
def gather_cols(params, indices, name=None):
    with tf.op_scope([params, indices], name, "gather_cols") as scope:
        # Check input
        params = tf.convert_to_tensor(params, name="params")
        indices = tf.convert_to_tensor(indices, name="indices")
        try:
            params.get_shape().assert_has_rank(2)
        except ValueError:
            raise ValueError('\'params\' must be 2D.')
        try:
            indices.get_shape().assert_has_rank(1)
        except ValueError:
            raise ValueError('\'params\' must be 1D.')
        
        # Define op
        p_shape = tf.shape(params)
        p_flat = tf.reshape(params, [-1])
        i_flat = tf.reshape(tf.reshape(tf.range(0, p_shape[0]) * p_shape[1],[-1, 1]) + indices, [-1])
        return tf.reshape(tf.gather(p_flat, i_flat),[p_shape[0], -1])

def get_theta(dim_baseline):
    m = np.eye(dim_baseline)
    m = np.add(m,.1)
    return m

class CustomKernelStatic(gpflow.kernels.Kernel):
    def __init__(self,input_dim, mysession=None,rhos=None,select_users=None, active_dims=None, ARD=None, name=None,baseline_indices = None,psi_indices=None,user_index = None,user_day_index = None,num_data_points = None):
        super().__init__(input_dim)
        
        
        ##make this a function
        theta = get_theta(len(baseline_indices)).reshape(1,len(baseline_indices),len(baseline_indices))
        
        #sigmau = tf.reshape(np.array([[1.0,0.1],[0.1,1.0]]),(1,2,2))
        sigmav = np.array([[10.0,0.0],[0.0,10.0]]).reshape(1,2,2)
        
        self.sigma_u = gpflow.Param(value = np.array([[1.0,0.1],[0.1,1.0]]).reshape(1,2,2),\
                                    transform=gpflow.transforms.DiagMatrix(2)(gpflow.transforms.positive))
            
                                    #tf.constant(np.array([[1.0,0.1],[0.1,1.0]]))
                                    
        self.sigma_theta = tf.constant(theta)
                                    #gpflow.Param(theta, transform=gpflow.transforms.DiagMatrix(6)(gpflow.transforms.positive),
                                    #                           dtype=gpflow.settings.float_type,fix_shape=True)
                                    #print(self.sigma_theta)
                                    #gpflow.Param(1.0, transform=gpflow.transforms.positive,
                                    #dtype=gpflow.settings.float_type)
                                    
        self.sigma_v =  gpflow.Param(sigmav, transform=gpflow.transforms.DiagMatrix(2)(gpflow.transforms.positive),
                                                                 dtype=gpflow.settings.float_type)
                                    
                                    
        self.noise_term = gpflow.Param(1.0, transform=gpflow.transforms.positive,
                                                                   dtype=gpflow.settings.float_type)
                                    
                                    #gpflow.Param(1.0, transform=gpflow.transforms.positive,
                                    #dtype=gpflow.settings.float_type)
        
                                    
        
        self.select_users = tf.constant(select_users)
                                    
        self.baseline_indices = baseline_indices
        self.psi_indices = psi_indices
                                    
        self.num_data_points = num_data_points
        self.user_index = user_index
        self.user_day_index = user_day_index
        self.mysession=mysession
        self.rhos = tf.constant(rhos)
##will it freak out if there is no parameter?


    @gpflow.params_as_tensors
    def rbf_custom(self, X, X2=None):
        #print(X)
        #print(X2)
        if X2 is None:
            X2=X
        return tf.exp(-tf.divide(tf.square(tf.subtract(X,X2)),tf.constant(1.0,dtype=tf.float64)))
        #return tf.constant(1.0,dtype=tf.float64)
    #tf.exp(-tf.subtract(X,X2) / float(2.2))
    
    
    
    @gpflow.params_as_tensors
    def K(self, X, X2=None):
        
        
        
        
        f_one = gather_cols(X, self.baseline_indices, name=None)
        f_one = tf.reshape(f_one,(1,self.num_data_points, len(self.baseline_indices)))
        g_one =gather_cols(X, self.psi_indices, name=None)
        g_one = tf.reshape(g_one,(1,self.num_data_points ,len(self.psi_indices)))
        user_id_one = gather_cols(X, [self.user_index], name=None)
        day_one = gather_cols(X, [self.user_day_index], name=None)
        
        
        
        #print(f_one.get_shape())
        
        if not X2 is None:
            print('called')
            f_two =gather_cols(X2,self.baseline_indices, name=None)
            #f_two = tf.reshape(f_two,(1,100,6))
            g_two = gather_cols(X2,  self.psi_indices, name=None)
            #g_two = tf.reshape(g_two,(1,100,2))
            user_id_two = gather_cols(X2, [self.user_index], name=None)
            day_two = gather_cols(X2, [self.user_day_index], name=None)
        
        
        
        else:
            #print('called')
            user_id_two = X[-2]
            day_two = day_one
            f_two = gather_cols(X, self.baseline_indices, name=None)
            g_two=gather_cols(X, self.psi_indices, name=None)
        
        
        
        
        #rho_term = self.rbf_custom(day_one,X2=day_two)
        #print('rho')
        #print(rho_term.get_shape())
        
        #baselines = tf.tensordot(tf.transpose(f_one),self.sigma_theta,axes=[[0],[1]])
        baselines = tf.reshape(tf.tensordot(f_one,self.sigma_theta,axes=[[2],[1]]),(self.num_data_points ,len(self.baseline_indices)))
        #print('baselines')
        #print(baselines.get_shape())
        #print('sigma theta')
        #print(self.sigma_theta)
        baselines = tf.tensordot(baselines,tf.transpose(f_two),axes=[[1],[0]])
        
        #print(baselines)
        #rhos = tf.constant(np.array([[self.rbf_custom(day_one[i], X2=day_two[j])[0]] \
        #for i, j in zip([i for i in range(100)], [j for j in range(100)])]))
        
        
        #effects = self.get_all_sigmas(self.sigma_v,g_one,g_two,day_one,day_two)
        #print('look at shape')t
        #print(baselines)
        #print('baselines')
        #print(baselines.get_shape())
        
        #effect_term = tf.tensordot(rho_term,self.sigma_v,axes=[[0],[0]])
        
        #user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))
        #effect_term = tf.add(user_term,effect_term)
        #print('eff')
        #print(effect_term.shape)
        
        #print('eff')
        #print(effects.shape)
        #user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))
        
        
        
        #print('eff')
        #print(effects.shape)
        
        ##important
        #print(self.sigma_u)
        #effects = tf.tensordot(tf.transpose(g_one),self.sigma_u[0],axes=[[0],[1]])
        #effects = tf.reshape(effects,(1,6,6))
        effects = tf.reshape(tf.tensordot(g_one,self.sigma_u,axes=[[2],[1]]),(self.num_data_points ,2))
        effects = tf.tensordot(effects,tf.transpose(g_two),axes = [[1],[0]])
        
        effects_one = tf.multiply(effects,self.select_users)
        
        
        effects = tf.reshape(tf.tensordot(g_one,self.sigma_v,axes=[[2],[1]]),(self.num_data_points ,2))
        #effects = tf.tensordot(tf.transpose(g_one),self.sigma_v[0],axes=[[0],[1]])
        
        
        effects = tf.tensordot(effects,tf.transpose(g_two),axes = [[1],[0]])
        effects_two = tf.multiply(effects,self.rhos)
        
        effects = tf.add(effects_one,effects_two)
        
        
        #print('eff')
        #print(effects.shape)
        result = tf.add(baselines,effects)
        
        #noise = 1000*np.eye(100)
        noise= tf.multiply(self.noise_term,tf.constant(np.eye(self.num_data_points )))
        noise = tf.reshape(noise,(1,self.num_data_points ,self.num_data_points ))
        
        result = tf.add(result,noise)
        result = tf.reshape(result,(self.num_data_points ,self.num_data_points ))
        #effect_term = tf.add(user_term,effect_term)
        #print('r')
        #print(result.shape)
        
        
        return result

In [7]:



class CustomKernel(gpflow.kernels.Kernel):
    def __init__(self,input_dim, mysession=None,rhos=None,select_users=None,
                 active_dims=None, ARD=None, name=None,baseline_indices = None,psi_indices=None,user_index = None,\
                user_day_index = None,num_data_points = None):
        super().__init__(input_dim)

        
        ##make this a function
        theta = get_theta(len(baseline_indices)).reshape(1,len(baseline_indices),len(baseline_indices))
        
        #sigmau = tf.reshape(np.array([[1.0,0.1],[0.1,1.0]]),(1,2,2))
        sigmav = np.array([[10.0,0.0],[0.0,10.0]]).reshape(1,2,2)                
        
        #self.sigma_u = gpflow.Param(value = np.array([[1.0,0.1],[0.1,1.0]]).reshape(1,2,2),\transform=gpflow.transforms.DiagMatrix(2)(gpflow.transforms.positive))
        #self.sigma_u1 = gpflow.Param(1.0, transform=gpflow.transforms.positive,dtype=gpflow.settings.float_type)
        #self.sigma_u2 = gpflow.Param(1.0, transform=gpflow.transforms.positive, dtype=gpflow.settings.float_type)
        #self.sigma_rho =gpflow.Param(1.0, transform=gpflow.transforms.Logistic(a=0,b=2), dtype=gpflow.settings.float_type)
        
        #tf.constant(np.array([[1.0,0.1],[0.1,1.0]]))
        
        self.sigma_theta = tf.constant(theta)
        #gpflow.Param(theta, transform=gpflow.transforms.DiagMatrix(6)(gpflow.transforms.positive),
         #                           dtype=gpflow.settings.float_type,fix_shape=True)
        #print(self.sigma_theta)
        #gpflow.Param(1.0, transform=gpflow.transforms.positive,
                                  #dtype=gpflow.settings.float_type)
        
        self.sigma_v =  gpflow.Param(sigmav, transform=gpflow.transforms.DiagMatrix(2)(gpflow.transforms.positive),dtype=gpflow.settings.float_type)
        

        #self.noise_term = gpflow.Param(1.0, transform=gpflow.transforms.positive,  dtype=gpflow.settings.float_type)
        
       
        
        sigmav = np.array([[10.0,0.0],[0.0,10.0]]).reshape(1,2,2)
        self.sigma_v =  gpflow.Param(sigmav,trainable=False, transform=gpflow.transforms.DiagMatrix(2)(gpflow.transforms.positive),dtype=gpflow.settings.float_type)
        
        
        self.sigma_u1 = tf.constant([1],dtype=tf.float64)
        self.sigma_u2 = tf.constant([1],dtype=tf.float64)
        self.sigma_rho =tf.constant([.2],dtype=tf.float64)
        self.sigma_theta = tf.constant(theta)
                                    #gpflow.Param(theta, transform=gpflow.transforms.DiagMatrix(6)(gpflow.transforms.positive),
                                    #                           dtype=gpflow.settings.float_type,fix_shape=True)
                                    #print(self.sigma_theta)
                                    #gpflow.Param(1.0, transform=gpflow.transforms.positive,
                                    #dtype=gpflow.settings.float_type)
                                    
        #self.sigma_v =  tf.constant(sigmav)
                                    
                                    
        self.noise_term = tf.constant([1],dtype=tf.float64)
        
        
        #gpflow.Param(1.0, transform=gpflow.transforms.positive,
                                  #dtype=gpflow.settings.float_type)
        self.mysession=mysession
        
        self.rhos = tf.constant(rhos)
        self.select_users = tf.constant(select_users)
        
        self.baseline_indices = baseline_indices
        self.psi_indices = psi_indices
        
        self.num_data_points = num_data_points
        self.user_index = user_index
        self.user_day_index = user_day_index
        
        ##will it freak out if there is no parameter?
        
    
    @gpflow.params_as_tensors
    def rbf_custom(self, X, X2=None):
        #print(X)
        #print(X2)
        if X2 is None:
             X2=X
        return tf.exp(-tf.divide(tf.square(tf.subtract(X,X2)),tf.constant(1.0,dtype=tf.float64)))
        return tf.constant(1.0,dtype=tf.float64)
    #tf.exp(-tf.subtract(X,X2) / float(2.2))
    

    
    @gpflow.params_as_tensors
    def K(self, X, X2=None):
 

      
        
        f_one = gather_cols(X, self.baseline_indices, name=None)
        f_one = tf.reshape(f_one,(1,self.num_data_points, len(self.baseline_indices)))
        g_one =gather_cols(X, self.psi_indices, name=None)
        g_one = tf.reshape(g_one,(1,self.num_data_points ,len(self.psi_indices)))
        user_id_one = gather_cols(X, [self.user_index], name=None)
        day_one = gather_cols(X, [self.user_day_index], name=None)



        #print(f_one.get_shape())
        
        if not X2 is None:
            print('called')
            f_two =gather_cols(X2,self.baseline_indices, name=None)
            #f_two = tf.reshape(f_two,(1,100,6))
            g_two = gather_cols(X2,  self.psi_indices, name=None)
            #g_two = tf.reshape(g_two,(1,100,2))
            user_id_two = gather_cols(X2, [self.user_index], name=None)
            day_two = gather_cols(X2, [self.user_day_index], name=None)
            
         
            
        else:
            #print('called')
            user_id_two = X[-2]
            day_two = day_one
            f_two = gather_cols(X, self.baseline_indices, name=None)
            g_two=gather_cols(X, self.psi_indices, name=None)
            
            
        sigma_u1 = self.sigma_u1
        print('sigma')
        t_one = sigma_u1
        t_two = tf.multiply(tf.math.sqrt(self.sigma_u1),tf.math.sqrt(self.sigma_u2))
        rho_term = tf.subtract(self.sigma_rho,1)
        t_two = tf.multiply(t_two,self.sigma_rho)
        t_three = self.sigma_u2
        t_four = t_two
        row_one = tf.stack([t_one, t_two],axis=0)
        row_two = tf.stack([t_three, t_four],axis=0)
        
        temp_sigma = tf.reshape(tf.stack([row_one,row_two]),[1,2,2])
        print(temp_sigma)
        #temp_sigma = tf.concat(tf.stack([t_one, t_two],axis=0),tf.stack([t_three, t_four],axis=0),axis=1)
        
        #print(temp_sigma)
        #rho_term = self.rbf_custom(day_one,X2=day_two)
        #print('rho')
        #print(rho_term.get_shape())
        
        #baselines = tf.tensordot(tf.transpose(f_one),self.sigma_theta,axes=[[0],[1]])
        baselines = tf.reshape(tf.tensordot(f_one,self.sigma_theta,axes=[[2],[1]]),(self.num_data_points ,len(self.baseline_indices)))
        print('baselines')
        print(baselines.get_shape())
        print('sigma theta')
        print(self.sigma_theta)
        baselines = tf.tensordot(baselines,tf.transpose(f_two),axes=[[1],[0]])
        
        print(baselines)
        #rhos = tf.constant(np.array([[self.rbf_custom(day_one[i], X2=day_two[j])[0]] \
                                     #for i, j in zip([i for i in range(100)], [j for j in range(100)])]))
        
        
        #effects = self.get_all_sigmas(self.sigma_v,g_one,g_two,day_one,day_two)
            #print('look at shape')t
            #print(baselines)
        #print('baselines')
        #print(baselines.get_shape())
        
        #effect_term = tf.tensordot(rho_term,self.sigma_v,axes=[[0],[0]])

        #user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))           
        #effect_term = tf.add(user_term,effect_term)
       #print('eff')
        #print(effect_term.shape)
        
        #print('eff')
        #print(effects.shape)
        #user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))           
        


        #print('eff')
        #print(effects.shape)
        
        ##important
        #print(self.sigma_u)
        #effects = tf.tensordot(tf.transpose(g_one),self.sigma_u[0],axes=[[0],[1]])
        #effects = tf.reshape(effects,(1,6,6))
        effects = tf.reshape(tf.tensordot(g_one,temp_sigma,axes=[[2],[1]]),(self.num_data_points ,2))
        effects = tf.tensordot(effects,tf.transpose(g_two),axes = [[1],[0]])
       
        effects_one = tf.multiply(effects,self.select_users)
        
        
        effects = tf.reshape(tf.tensordot(g_one,self.sigma_v,axes=[[2],[1]]),(self.num_data_points ,2))
        #effects = tf.tensordot(tf.transpose(g_one),self.sigma_v[0],axes=[[0],[1]])
       
        
        effects = tf.tensordot(effects,tf.transpose(g_two),axes = [[1],[0]])
        effects_two = tf.multiply(effects,self.rhos)
        
        effects = tf.add(effects_one,effects_two)
   
        
        #print('eff')
        #print(effects.shape)
        result = tf.add(baselines,effects)
        
        #noise = 1000*np.eye(100)
        noise= tf.multiply(self.noise_term,tf.constant(np.eye(self.num_data_points )))
        noise = tf.reshape(noise,(1,self.num_data_points ,self.num_data_points ))
        
        result = tf.add(result,noise)
        result = tf.reshape(result,(self.num_data_points ,self.num_data_points ))
        #effect_term = tf.add(user_term,effect_term)
        print('r')
        print(result.shape)
        
        
        return result
       
        
        #self.variance * tf.minimum(X, tf.transpose(X2))


In [8]:
##need to reformat history
def put_x_together(h):
    X = []
    
    for i in range(len(h.states)):
        state_vector = h.states[i]
        state_vector.append(float(h.users[i]))
        state_vector.append(float(h.day_function[i]))
        X.append(state_vector)
    return X


In [11]:
#history = gh.get_history(8,100,5)


In [12]:
#X = put_x_together(history)


In [11]:
#X = ad['processed_history']
X = np.array(ad['processed_history'].tolist())


In [10]:
len(X)

16060

In [12]:
users = np.array([[float(X[i][8]==X[j][8]) for j in range(len(X))] for i in range(len(X))])


In [13]:
def rbf_custom_np( X, X2=None):
        #print(X)
        #print(X2)
        if X2 is None:
             X2=X
        return math.exp(-((X-X2)**2)/1.0)
        #tf.divide(tf.square(-tf.subtract(X,X2)),tf.constant(1.0,dtype=tf.float64)))
rdayone = [x[9] for x in X]
rdaytwo = rdayone
rhos = np.array([[rbf_custom_np( rdayone[i], X2=rdaytwo[j]) for j in range(len(X))] for i in range(len(X))])
#


In [14]:
y = np.array([[float(r)] for r in ad['steps'].tolist()])


In [15]:
len(y)

8030

In [16]:
sess = tf.Session()
k =  CustomKernel(10,mysession=sess,rhos=rhos,select_users=users,baseline_indices=[0,1,2,3,4,5,6],\
                  psi_indices=[0,7],user_day_index=9,user_index=8,num_data_points=len(X))

m = gpflow.models.GPR(X,y, kern=k)
m.initialize(session=sess)

sigma
Tensor("GPR-8e0dfc0f-7/likelihood_1/Reshape_2:0", shape=(1, 2, 2), dtype=float64)
baselines
(8030, 7)
sigma theta
Tensor("Const_4:0", shape=(1, 7, 7), dtype=float64)
Tensor("GPR-8e0dfc0f-7/likelihood_1/Tensordot_1:0", shape=(8030, ?), dtype=float64)


r
(8030, 8030)


In [50]:
gpflow.train.ScipyOptimizer().minimize(m,session=sess)


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 1463.597514
  Number of iterations: 13
  Number of functions evaluations: 14


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 1463.597514
  Number of iterations: 13
  Number of functions evaluations: 14


In [33]:
m.as_pandas_table()

class prior       transform  trainable  \
GPR/kern/sigma_v         Parameter  None  DiagMatrix +ve      False   
GPR/likelihood/variance  Parameter  None             +ve       True   

                             shape  fixed_shape                         value  
GPR/kern/sigma_v         (1, 2, 2)         True  [[[10.0, 0.0], [0.0, 10.0]]]  
GPR/likelihood/variance         ()         True                           1.0

In [20]:
dir(m)

['X',
 'Y',
 '_Parentable__global_index',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_build',
 '_build_likelihood',
 '_build_objective',
 '_build_predict',
 '_build_prior',
 '_clear',
 '_define_name',
 '_gen_index',
 '_get_node',
 '_index',
 '_is_param_like',
 '_likelihood_tensor',
 '_name',
 '_objective',
 '_parent',
 '_prior_tensor',
 '_read_index',
 '_replace_node',
 '_repr_html_',
 '_set_node',
 '_tensor_mode_parameter',
 '_update_node',
 'anchor',
 'as_pandas_table',
 'assign',
 'build',
 'build_objective',
 'childname',
 'children',
 'clear',
 'c

In [19]:
type(term)

method

In [59]:

term= m.kern.K(X,X2=X)


called


sigma
Tensor("Reshape_75:0", shape=(1, 2, 2), dtype=float64)
baselines
(1000, 7)
sigma theta
Tensor("Const_78:0", shape=(1, 7, 7), dtype=float64)
Tensor("Tensordot_49:0", shape=(1000, 1000), dtype=float64)
r
(1000, 1000)


In [60]:
trm = term.eval(session=sess)


In [54]:
trm

array([[27.37906886, 26.37906886, 26.37906886, ...,  1.1100741 ,
         1.1100741 , -1.14521405],
       [26.37906886, 27.37906886, 26.37906886, ...,  1.1100741 ,
         1.1100741 , -1.14521405],
       [26.37906886, 26.37906886, 27.37906886, ...,  1.1100741 ,
         1.1100741 , -1.14521405],
       ...,
       [ 1.1100741 ,  1.1100741 ,  1.1100741 , ...,  4.01613104,
         3.01613104,  0.76492324],
       [ 1.1100741 ,  1.1100741 ,  1.1100741 , ...,  3.01613104,
         4.01613104,  0.76492324],
       [-1.14521405, -1.14521405, -1.14521405, ...,  0.76492324,
         0.76492324, 11.99014601]])

In [25]:
trm


array([[27.37906886, 26.37906886, 26.37906886, ...,  1.1100741 ,
         1.1100741 , -1.14521405],
       [26.37906886, 27.37906886, 26.37906886, ...,  1.1100741 ,
         1.1100741 , -1.14521405],
       [26.37906886, 26.37906886, 27.37906886, ...,  1.1100741 ,
         1.1100741 , -1.14521405],
       ...,
       [ 1.1100741 ,  1.1100741 ,  1.1100741 , ...,  4.01613104,
         3.01613104,  0.76492324],
       [ 1.1100741 ,  1.1100741 ,  1.1100741 , ...,  3.01613104,
         4.01613104,  0.76492324],
       [-1.14521405, -1.14521405, -1.14521405, ...,  0.76492324,
         0.76492324, 11.99014601]])

In [63]:
def get_RT(y,X,sigma_theta):
    
    to_return = [y[i]-np.dot(X[i][0:6],np.ones(6)) for i in range(len(X))]
    return np.array([i[0] for i in to_return])

In [64]:
def create_H():
    return np.transpose(np.array([[1,0,0,0,0,0],[0,0,1,0,1,0]]))
    
    

In [65]:
def get_MT(new_x,sigma_theta,sigma_u,sigma_v,history,H):
    
    psi = [new_x[0],new_x[7]]
    user_id = new_x[8]
    day_id = new_x[9]
    
    to_return = [[] for i in range(history.shape[0])]
    
    for x_old_i in range(len(history)):
        x_old = history[x_old_i]
        phi = x_old[1:7]
       
        old_user_id = x_old[8]
        old_day_id = x_old[9]
        
        inner = float(old_user_id==user_id)*sigma_u+rbf_custom_np(day_id,old_day_id)*sigma_v
        #print(inner.shape)
        #print(H.shape)
        inner = np.dot(H,inner)
        #print(inner.shape)
        inner = np.dot(inner,np.transpose(H))
        #print(inner.shape)
        inner = np.add(sigma_theta,inner)
        
        
        
        
        term = np.dot(np.transpose(phi),inner)
        to_return[x_old_i]=[i for i in term]
    return to_return


In [68]:
    sigma_theta = np.array([[1.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 1.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 1.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 1.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 1.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 1.1]])
    sigma_u = np.array([[m.kern.sigma_u1.value,m.kern.sigma_u1.value**.5*m.kern.sigma_u2.value**.5*m.kern.sigma_rho.value],\
                        [m.kern.sigma_u1.value**.5*m.kern.sigma_u2.value**.5*m.kern.sigma_rho.value,m.kern.sigma_u2.value]])
    #np.array([[1.0,0.1],[0.1,1.0]])
    
    sigma_v = m.kern.sigma_v.value
    #np.array([[10.0,0.0],[0.0,10.0]])
    #np.array([[10.0,0.0],[0.0,10.0]])
    
    

In [69]:
sigma_u

array([[1.00461865e-06, 2.56413450e-19],
       [2.56413450e-19, 1.00461865e-06]])

In [70]:

    
def write_all():
    
    df = pd.DataFrame(X,columns=['f_1','f_2','f_3','f_4','f_5','f_6','g_1','g_2','user_id','user_day_id'])
    path = 'X_data.feather'
    feather.write_dataframe(df, path)
    df_rewards = pd.DataFrame(y,columns=['reward'])
    path = 'y_data.feather'
    feather.write_dataframe(df_rewards, path)
    r_adjusted = get_RT(y,X,sigma_theta)    
    df_rewardsa = pd.DataFrame(r_adjusted,columns=['reward_centered'])
    path = 'adjusted_R.feather'
    feather.write_dataframe(df_rewardsa , path)
    all_MS = [get_MT(x,sigma_theta,sigma_u.reshape(2,2),sigma_v.reshape(2,2),X,create_H()) for x in X[:1]]
    df_rewardsa = pd.DataFrame(all_MS[0],columns=['x_{}'.format(i) for i in range(6)])
    path = 'M_one.feather'
    feather.write_dataframe(df_rewardsa , path)
    
    df_rewardsa = pd.DataFrame(trm,columns=['u_{}'.format(i) for i in range(1000)])
    path = 'covar_from_kernel_on_train.feather'
    feather.write_dataframe(df_rewardsa , path)
    
    
    df_rewardsa = pd.DataFrame(sigma_theta,columns=['i_{}'.format(i) for i in range(6)])
    path = 'sigma_theta.feather'
    feather.write_dataframe(df_rewardsa , path)
    
    df_rewardsa = pd.DataFrame(sigma_v.reshape(2,2),columns=['i_{}'.format(i) for i in range(2)])
    path = 'sigma_v.feather'
    feather.write_dataframe(df_rewardsa , path)
    
    df_rewardsa = pd.DataFrame(sigma_u.reshape(2,2),columns=['i_{}'.format(i) for i in range(2)])
    path = 'sigma_u.feather'
    feather.write_dataframe(df_rewardsa , path)
    
    df_rewardsa = pd.DataFrame([m.kern.noise_term.value],columns=['i_{}'.format(i) for i in range(1)])
    path = 'noise.feather'
    feather.write_dataframe(df_rewardsa , path)
    
    
    

In [76]:
def get_middle_term(X,user_id,user_study_day,cov,noise_term,M,adjusted_rewards):
    #M = get_M(global_params,user_id,user_study_day,history[0])
    
    ##change this to be mu_theta
    ##is it updated?  the current mu_theta?
    #adjusted_rewards =[history[1][i]-np.dot(history[0][i][0:6],np.ones(6)) for i in range(len(history[0]))]
    
    noise = noise_term * np.eye(X.shape[0])
    middle_term = np.add(cov,noise)
    
    middle_term = np.dot(M.T,np.linalg.inv(middle_term))
    print(M.shape)
    middle_term = np.dot(middle_term,adjusted_rewards)
    return middle_term

In [85]:
def get_post_sigma(H,cov,sigma_u,sigma_v,noise_term,M):
    #M = get_M(global_params,user_id,user_study_day,history[0])
    
    ##change this to be mu_theta
    ##is it updated?  the current mu_theta?
    #adjusted_rewards =[history[1][i]-np.dot(history[0][i][0:6],np.ones(6)) for i in range(len(history[0]))]
    
    
    
    first_term = np.add(sigma_u,sigma_v)
    first_term = np.dot(H,first_term)
    first_term = np.dot(first_term,H.T)
    
    
    noise = noise_term * np.eye(X.shape[0])
    middle_term = np.add(cov,noise)
    
    middle_term = np.dot(M.T,np.linalg.inv(middle_term))

    middle_term = np.dot(middle_term,M)
    
    last = np.add(sigma_theta,first_term)
    last = np.subtract(last,middle_term)
    
    return middle_term

In [ ]:

##test get M    
def get_to_test():
    

    r_adjusted = get_RT(y,X,sigma_theta)    
 
    all_MS = [get_MT(x,sigma_theta,sigma_u.reshape(2,2),sigma_v.reshape(2,2),X,create_H()) for x in X[:1]]

       

In [111]:
m.kern.noise.value.shape

(1, 100, 100)

In [71]:
write_all()

In [114]:
import numpy as np

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

In [70]:
is_pos_def(m.kern.sigma_theta.value.reshape(2,2))

NameError: name 'is_pos_def' is not defined

In [12]:
all_MS = [get_MT(x,sigma_theta,sigma_u.reshape(2,2),sigma_v.reshape(2,2),X,create_H()) for x in X[:1]]


In [16]:
np.array(all_MS[0]).shape

(1000, 6)

In [82]:
#r_adjusted = np.array([i[0] for i in get_RT(y,X,sigma_theta)])
r_adjusted=get_RT(y,X,sigma_theta)

In [83]:
r_adjusted.shape

(1000,)

In [77]:
test = get_middle_term(X,1,2,trm,noise_term,np.array(all_MS)[0],r_adjusted)

(1000, 6)


In [88]:
ttwo = get_post_sigma(create_H(),trm,sigma_u,sigma_v,m.kern.noise_term.value,np.array(all_MS)[0])

In [93]:
ttwo

array([[ 1.23366992e+04, -9.13374931e-01,  3.58906263e-01,
         3.95195294e-01,  8.84852494e-03,  3.68929939e-01],
       [-9.13374931e-01,  1.09795255e+00,  9.98811741e-02,
         9.99937166e-02,  1.00095602e-01,  9.98942771e-02],
       [ 3.58906263e-01,  9.98811741e-02,  3.33013432e-02,
         9.95990132e-02,  1.00183018e-01,  9.95890013e-02],
       [ 3.95195294e-01,  9.99937166e-02,  9.95990132e-02,
         1.09613267e+00,  1.00979414e-01,  9.65749886e-02],
       [ 8.84852494e-03,  1.00095602e-01,  1.00183018e-01,
         1.00979414e-01,  1.09861268e+00,  1.01167021e-01],
       [ 3.68929939e-01,  9.98942771e-02,  9.95890013e-02,
         9.65749886e-02,  1.01167021e-01,  1.09587620e+00]])

In [92]:
[j[-3:] for j in ttwo[-3:]]

[array([1.09613267, 0.10097941, 0.09657499]),
 array([0.10097941, 1.09861268, 0.10116702]),
 array([0.09657499, 0.10116702, 1.0958762 ])]

In [38]:
noise_term = m.kern.noise_term.value

In [118]:
#one = tf.constant([0,0])
#two = tf.constant([1,1])
inp= tf.constant([[1.0, 2, 3], [4, 5, 6.0], [7.0, 8, 9]])
tf.norm([inp,inp],ord=2)


<tf.Tensor 'norm_9/Squeeze:0' shape=() dtype=float32>

In [126]:
df = pd.DataFrame(X,columns=['f_1','f_2','f_3','f_4','f_5','f_6','g_1','g_2','user_id','user_day_id'])
path = 'X_data.feather'
feather.write_dataframe(df, path)
df_rewards = pd.DataFrame(y,columns=['reward'])
path = 'y_data.feather'
feather.write_dataframe(df_rewards, path)

In [113]:
#path = 'adjusted_R.feather'
#feather.write_dataframe(df, path)
df_rewardsa = pd.DataFrame(r_adjusted,columns=['reward_centered'])
path = 'adjusted_R.feather'
feather.write_dataframe(df_rewardsa , path)

In [136]:
trm

array([[31151.07511374, 33229.2577411 , 31246.14996826, ...,
        29977.84463435, 29732.87325126, 29964.07900045],
       [33229.2577411 , 37610.47091257, 34427.60527294, ...,
        33039.88677868, 32762.35846839, 33014.83012101],
       [31246.14996826, 34427.60527294, 33373.76040804, ...,
        31066.54338706, 30805.94330263, 31044.38989066],
       ...,
       [29977.84463435, 33039.88677868, 31066.54338706, ...,
        30794.9554853 , 29551.69047504, 29791.74692022],
       [29732.87325126, 32762.35846839, 30805.94330263, ...,
        29551.69047504, 30303.45553549, 29541.5013214 ],
       [29964.07900045, 33014.83012101, 31044.38989066, ...,
        29791.74692022, 29541.5013214 , 30765.70883504]])

In [137]:
import pandas

In [141]:
from rpy2.robjects import numpy2ri
import rpy2.robjects as robjects
write_csv = robjects.r('write.csv')
write_csv(all_MS,'filename.csv')
#r("save(foo, file='here.gzip', compress=TRUE)"

rpy2.rinterface.NULL

In [132]:
import pickle
with open('temp_M.pkl','wb') as f:
    pickle.dump(all_MS,f)

In [137]:
df_rewardsa = pd.DataFrame(trm,columns=['u_{}'.format(i) for i in range(100)])
path = 'covar_from_kernel_on_train.feather'
feather.write_dataframe(df_rewardsa , path)


In [138]:
learned_s_theta = m.kern.sigma_theta.value
df_rewardsa = pd.DataFrame(learned_s_theta,columns=['i_{}'.format(i) for i in range(6)])
path = 'sigma_theta.feather'
feather.write_dataframe(df_rewardsa , path)


In [139]:
learned_s_u = m.kern.sigma_u.value
df_rewardsa = pd.DataFrame(learned_s_u,columns=['i_{}'.format(i) for i in range(2)])
path = 'sigma_u.feather'
feather.write_dataframe(df_rewardsa , path)

In [140]:
learned_s_v = m.kern.sigma_v.value
df_rewardsa = pd.DataFrame(learned_s_v,columns=['i_{}'.format(i) for i in range(2)])
path = 'sigma_v.feather'
feather.write_dataframe(df_rewardsa , path)


In [125]:
pp

[[1, 1, 2, 2, 2, 3, 3, 3], [1, 1, 2, 2, 2, 3, 3, 3]]

In [32]:
    def rbf_custom( X, X2=None):
        #print(X)
        #print(X2)
        if X2 is None:
             X2=X
        return tf.exp(tf.divide(tf.square(-tf.subtract(X,X2)),tf.constant(1.0,dtype=tf.float64)))
        return tf.constant(1.0,dtype=tf.float64)

In [519]:
tx = tf.constant(X)
day_two = gather_cols(tx, [6], name=None)
z= rbf_custom(day_two,day_two)

In [520]:
z

<tf.Tensor 'Exp_756:0' shape=(100, 1) dtype=float64>

In [521]:
def get_all_sigmas(sigma_v,g_one,g_two,day_one,day_two):
    #sigmax = [[] for i in range(100)]
    #tf.placeholder(tf.float64, shape=(100, 2,2))
    
            
        #print('eff')
        #print(effects.shape)
        #user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))           
        


        #print('eff')
        #print(effects.shape)
        #effects = tf.tensordot(effects,tf.transpose(g_two),axes = [[1],[0]])
    
    get_g = g_one
    get_g_two = g_two
    to_return = [[] for i in range(get_g.get_shape()[0])]
    for i in range(get_g.get_shape()[0]):
        temp = [[] for i in range(get_g_two.get_shape()[0])]
        for j in range( get_g_two.get_shape()[0]):
            rho = rbf_custom(day_one[i], X2=day_two[j])
            #print(sess.run(rho))
            val = sess.run(rho)[0]
        #tf.scalar_mul(val,sigma_v)
            #print(val)
            this_sigma = tf.scalar_mul(val,sigma_v)
            #print(this_sigma)
            effects = tf.tensordot(get_g[i],this_sigma,axes=[[0],[1]])
            effects = tf.tensordot(effects,get_g_two[j],axes = [[0],[0]])
            temp[j]=effects
        to_return[i]=temp
        
    return to_return

In [465]:
(sess.run(z[0])[0])

1.0

In [28]:
        day_one = gather_cols(X, [6], name=None)
        f_one = gather_cols(X, [0,1,2], name=None)
        g_one = gather_cols(X, [3,4], name=None)

In [38]:
rbf_custom(day_one[0], X2=day_two[0])[0]

<tf.Tensor 'strided_slice_100004:0' shape=() dtype=float64>

In [30]:
g_two=g_one
day_two=day_one

In [49]:
ts = tf.stack([day_one,day_two],axis=1)

In [54]:
elems = ts

alternate = tf.map_fn(lambda x: rbf_custom(x[0], X2=x[1]), elems, dtype=(tf.tensor))

AttributeError: module 'tensorflow' has no attribute 'tensor'

In [72]:
rhos.shape

(100, 100)

In [433]:
sigma_theta =tf.constant(np.array([[100.0,10.0,10.0],[10.0,100.0,10.0],\
                                    [10.0,10.0,100.0]]))
sigma_v =  tf.constant(np.array([[100.0,0.0],[0.0,100.0]]))
sigma_u =  tf.constant(np.array([[100.0,10.0],[10.0,100.0]]))

In [199]:
user_id_one = X_one[-2]
day_one = X_one[-1]
f_one = X_one[0:3]
g_one = X_one[3:5]

user_id_two = X_two[-2]
day_two = X_two[-1]
f_two = X_two[0:3]
g_two = X_two[3:5]
        
        
rho_term = rbf_custom(day_one,X2=day_two).eval(session=sess) 
#print(sess.run(rho_term))
print('rho')
#print(tf.shape(f_one))
print(rho_term)
print(f_one.shape)
print(f_one)
#print(rho_term.get_shape()[1])
            #rho_term = tf.constant(np.array([1.0]))
       
baselines = tf.tensordot(tf.transpose(f_one),sigma_theta,axes=[[0],[0]])
print('baselines')
print(baselines.get_shape())
baselines = tf.tensordot(tf.transpose(baselines),f_two,axes=[[0],[0]])
            #print('look at shape')
            #print(baselines)
print('baselines')
print(baselines.get_shape())
print(baselines.eval(session=sess))
#if user_id_one==user_id_two:
#    user_term = self.sigma_u
#else:
#print('user')
#print('user_term')

            #print(self.sigma_v.shape)
            #print(rho_term.shape)
effect_term = tf.scalar_mul(rho_term,sigma_v)
#print('effect term')
#print(effect_term.shape)
#print('user term')
#print(user_term.shape)
user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))           
effect_term = tf.add(user_term,effect_term)
#print('eff')
#print(effect_term.shape)
#print('g')
#print(g_one.shape)
effects = tf.tensordot(tf.transpose(g_one),effect_term,axes=[[0],[1]])
print('eff')
print(effects.shape)
effects = tf.tensordot(tf.transpose(effects),g_two,axes = [[0],[0]])
print('eff')
print(effects.shape)
result = tf.add(baselines,effects)
print('res')
print(result.shape)
print(result.eval(session=sess))

rho
1.0
(3,)
Tensor("strided_slice_206:0", shape=(3,), dtype=float64)
baselines
(3,)
baselines
()
107826.87763336947
eff
(2,)
eff
()
res
()
169293.14642401805


In [222]:
X = np.vstack([np.random.randn(10,2) + [2,2],
               np.random.randn(10,2) + [-2,2],
               np.random.randn(10,2) + [2,-2]])

In [223]:
X.shape

(30, 2)

In [ ]:
 #.eval(session=sess) 
            #print(sess.run(rho_term))
            print('rho')
            #print(tf.shape(f_one))
            print(rho_term)
            print(f_one.shape)
            print(f_one)
            #print(rho_term.get_shape()[1])
            #rho_term = tf.constant(np.array([1.0]))
       
            baselines = tf.tensordot(tf.transpose(f_one),self.sigma_theta,axes=[[0],[0]])
            print('baselines')
            print(baselines.get_shape())
            baselines = tf.tensordot(tf.transpose(baselines),f_two,axes=[[0],[0]])
            #print('look at shape')
            #print(baselines)
            print('baselines')
            print(baselines.get_shape())
           
            #if user_id_one==user_id_two:
            #    user_term = self.sigma_u
            #else:
            #print('user')
            #print('user_term')

            #print(self.sigma_v.shape)
            #print(rho_term.shape)
            effect_term = tf.tensordot(rho_term,self.sigma_v,axes=[[0],[0]])
                #print('effect term')
            #print(effect_term.shape)
            #print('user term')
            #print(user_term.shape)
            user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))           
            effect_term = tf.add(user_term,effect_term)
            #print('eff')
            #print(effect_term.shape)
            #print('g')
            #print(g_one.shape)
            effects = tf.tensordot(tf.transpose(g_one),effect_term,axes=[[0],[1]])
            print('eff')
            print(effects.shape)
            effects = tf.tensordot(tf.transpose(effects),g_two,axes = [[0],[0]])
            print('eff')
            print(effects.shape)
            result = tf.add(baselines,effects)
            print('res')
            print(result.shape)
        

In [4]:
import pickle
with open('temp_M.pkl','rb') as f:
    M=pickle.load(f)

In [12]:
import numpy as np
np.array(M[0]).shape
import pandas as pd

In [142]:
df_rewardsa = pd.DataFrame(all_MS[0],columns=['p_1','p_2','p_3','p_4','p_5','p_6'])
path = 'M0.feather'
feather.write_dataframe(df_rewardsa , path)

In [141]:
all_MS

[[[3.182050526733636,
   0.3967589131552818,
   6.129668130181115,
   0.4108408583014009,
   6.1199090701103325,
   0.41339359993659147],
  [2.735618497832924,
   0.4546450092332019,
   5.964244510574703,
   0.42449240150178946,
   5.9130813021233655,
   0.42733408838021547],
  [2.9705708116190817,
   0.4317596370968596,
   5.71146624848382,
   0.4051573511460121,
   5.689708344993629,
   0.416677634192122],
  [3.0534396947159417,
   0.4526882426350584,
   6.228919737478516,
   0.47597293743519464,
   6.218229396267928,
   0.4402007249318048],
  [1.2716599060579667,
   0.44551685717372563,
   2.3709108762878204,
   0.42937704784495495,
   2.363934517174844,
   0.408201566164266],
  [1.7289210611382186,
   0.42393436741566953,
   2.821302626418396,
   0.4366285336886993,
   2.7778015434508023,
   0.43261882216295106],
  [0.4727951688559979,
   0.4553525937882923,
   0.5447621970644432,
   0.4225580434776346,
   0.530102034404852,
   0.44287845716929924],
  [0.8320472022148684,
   0.4064

In [49]:
xl = tf.constant([1,2])
yl = tf.constant([2,3])
sess.run(tf.stack([xl,yl],axis=0))

array([[1, 2],
       [2, 3]], dtype=int32)